In [1]:
# Import pandas library. This will be used for reading and transforming the raw data.
import pandas as pd
import sqlalchemy

In [ ]:
# https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting
# Import flat file and store it into a pandas dataframe.
df = pd.read_csv('sales_data.csv', delimiter=',')

# Display the top 5 rows.
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


Rename the columns so that there are no spaces or special characters.

In [3]:
# Create a list of preferred column names and use the method .columns for renaming.
df.columns = ['rowId', 'orderId', 'orderDate', 'shipDate','shipMode', 'customerId', 'customerName', 'segment', 'country',
              'city', 'state', 'postalCode', 'region', 'productId', 'category', 'subCategory', 'productName', 'salesAmount']

# Display the top 5 rows.
df.head()

,rowId,orderId,orderDate,shipDate,shipMode,customerId,customerName,segment,country,city,state,postalCode,region,productId,category,subCategory,productName,salesAmount
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


Check for duplicates with the assumption that the combination of rowId and orderId creates a unique key.

In [4]:
# Create a variable that contains a list of all the columns except for rowId and orderId.
dupCheckColumns = ['orderDate', 'shipDate','shipMode', 'customerId', 'customerName', 'segment', 'country',
              'city', 'state', 'postalCode', 'region', 'productId', 'category', 'subCategory', 'productName', 'salesAmount']

In [5]:
# Check how many rows are duplicated.
print(df.duplicated(subset=dupCheckColumns).sum())

1


In [6]:
# Display the duplicate rows.
display(df[df.duplicated(subset=dupCheckColumns, keep=False)])

,rowId,orderId,orderDate,shipDate,shipMode,customerId,customerName,segment,country,city,state,postalCode,region,productId,category,subCategory,productName,salesAmount
3405,3406,US-2015-150119,23/04/2015,27/04/2015,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229.0,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372
3406,3407,US-2015-150119,23/04/2015,27/04/2015,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229.0,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372


In [7]:
# Check how many rows are in the dataframe.
df.shape

(9800, 18)

In [8]:
# Filter out the duplicated row.
df = df[~((df['rowId'] == 3407) & (df['orderId'] == 'US-2015-150119'))]

# Check how many rows are in the dataframe after removing the duplicate row.
df.shape

(9799, 18)

In [9]:
# Fix the rowId column
df['rowId'] = df.index
df.head()

,rowId,orderId,orderDate,shipDate,shipMode,customerId,customerName,segment,country,city,state,postalCode,region,productId,category,subCategory,productName,salesAmount
0,0,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,2,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,3,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [10]:
# Validate if the duplicated row has been removed
display(df[df['orderId'] == 'US-2015-150119'])

,rowId,orderId,orderDate,shipDate,shipMode,customerId,customerName,segment,country,city,state,postalCode,region,productId,category,subCategory,productName,salesAmount
3405,3405,US-2015-150119,23/04/2015,27/04/2015,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229.0,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372
3407,3407,US-2015-150119,23/04/2015,27/04/2015,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229.0,East,OFF-BI-10000145,Office Supplies,Binders,Zipper Ring Binder Pockets,7.488
3408,3408,US-2015-150119,23/04/2015,27/04/2015,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229.0,East,FUR-FU-10002191,Furniture,Furnishings,G.E. Halogen Desk Lamp Bulbs,22.336


Check for null values.

In [11]:
print(df.isnull().sum())

rowId            0
orderId          0
orderDate        0
shipDate         0
shipMode         0
customerId       0
customerName     0
segment          0
country          0
city             0
state            0
postalCode      11
region           0
productId        0
category         0
subCategory      0
productName      0
salesAmount      0
dtype: int64


Check the data types.

In [12]:
print(df.dtypes)

rowId             int64
orderId          object
orderDate        object
shipDate         object
shipMode         object
customerId       object
customerName     object
segment          object
country          object
city             object
state            object
postalCode      float64
region           object
productId        object
category         object
subCategory      object
productName      object
salesAmount     float64
dtype: object


Transform values into the proper format and data type.

In [13]:
# Convert orderDate and shipDate to date data type. This will also change the format to yyyy-MM-dd.
df['orderDate'] = pd.to_datetime(df['orderDate'], format='%d/%m/%Y')
df['shipDate'] = pd.to_datetime(df['shipDate'], format='%d/%m/%Y')

# Change postalCode data type to integer and assign the value 0 to nulls.
df['postalCode'] = df['postalCode'].fillna(0).astype(int)
# Change postalCode data type back to string.
df["postalCode"] = df["postalCode"].astype(str)

# Limit salesAmount to 2 decimal places
df['salesAmount'] = df['salesAmount'].apply(lambda x: round(x,2))

# Display the top 5 rows.
df.head()

,rowId,orderId,orderDate,shipDate,shipMode,customerId,customerName,segment,country,city,state,postalCode,region,productId,category,subCategory,productName,salesAmount
0,0,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
2,2,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62
3,3,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58
4,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.37


**Load the dataframe into the database**

In [14]:
# Set database variables
server = 'DESKTOP-JVDHPJV\SQLEXPRESS'
database = 'PROJECT_ALBATROSS_2025'
driver = 'ODBC Driver 17 for SQL Server'

In [15]:
# Set connection string
connection_string = f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver={driver}"

In [16]:
# Create engine
engine = sqlalchemy.create_engine(connection_string)

In [18]:
# Load df into database table
table_name = 'RAW_sales'
df.to_sql(table_name, engine, if_exists='replace',index=False)

print(f"Loading successful.")

Loading successful.
